# Train SqueezeNet model on tif for multi-label classification of tag = 0 (artisanal mining)

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [3]:
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [4]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

from data_utils import get_id_type_list_from_df, OUTPUT_PATH, GENERATED_DATA, to_set, RESOURCES_PATH
from training_utils import classification_train as train, classification_validate as validate
from training_utils import exp_decay, step_decay, get_high_zoom_imgaug_seq

Using TensorFlow backend.


In [5]:
from models.squeezenet_tif_classification import get_squeezenet_on_tif

from sklearn.model_selection import KFold
from data_utils import equalized_data_classes, unique_tags, TRAIN_ENC_CL_CSV
from data_utils import load_pretrained_model, get_label
from data_utils import DataCache

from xy_providers import tif_image_label_provider
from models.keras_metrics import binary_crossentropy_with_false_negatives


# Setup configuration

seed = 2017
np.random.seed(seed)

cache = DataCache(6000)  # !!! CHECK BEFORE LOAD TO FLOYD

tag = equalized_data_classes[0][0]

mask = TRAIN_ENC_CL_CSV[tag] > 0

trainval_id_type_list = get_id_type_list_from_df(TRAIN_ENC_CL_CSV[mask], 'Train_tif')

# ### ADD GENERATED IMAGES
# from glob import glob
# from data_utils import GENERATED_DATA, to_set, get_label

# gen_train_files = glob(os.path.join(GENERATED_DATA, "train", "tif", "*.tif"))
# gen_train_ids = [s[len(os.path.join(GENERATED_DATA, "train", "tif"))+1+len('gen_train_'):-4] for s in gen_train_files]
# gen_id_type_list = [(image_id, "Generated_Train_tif") for image_id in gen_train_ids]
# class_index_gen_train_ids = [id_type for id_type in gen_train_ids if np.sum(get_label(*gen_id_type_list[0], class_index=class_index)) > 0]
# class_index_gen_id_type_list = [(image_id, "Generated_Train_tif") for image_id in class_index_gen_train_ids]
# trainval_id_type_list = trainval_id_type_list + class_index_gen_id_type_list
# ### ADD GENERATED IMAGES


tags = list(unique_tags)
tags.remove(tag)

n_other_samples = int(len(trainval_id_type_list) * 1.0 / len(tags))

for t in tags:
    mask = TRAIN_ENC_CL_CSV[t] > 0    
    id_type_list = np.array(get_id_type_list_from_df(TRAIN_ENC_CL_CSV[mask], 'Train_tif'))
    id_type_list = list(to_set(id_type_list) - to_set(trainval_id_type_list))
    np.random.shuffle(id_type_list)
    trainval_id_type_list.extend(id_type_list[:n_other_samples])

print(len(trainval_id_type_list), len(to_set(trainval_id_type_list)))

675 675


In [6]:
from data_utils import get_caption
trainval_id_type_list[0], get_caption(*trainval_id_type_list[0])

(('57', 'Train_tif'), 'artisinal_mine bare_ground clear primary water')

In [7]:
params = {
    'seed': seed,

    'xy_provider': tif_image_label_provider,

    'network': get_squeezenet_on_tif,
    'optimizer': 'nadam',
    'loss': 'binary_crossentropy',
    'nb_epochs': 25,    # !!! CHECK BEFORE LOAD TO FLOYD
    'batch_size': 32,  # !!! CHECK BEFORE LOAD TO FLOYD

    'normalize_data': True,
    'normalization': '',

    'train_seq': get_high_zoom_imgaug_seq,
    
    'image_size': (256, 256),

    'tag': tag,

    # Learning rate scheduler
    'lr_kwargs': {
        'lr': 0.01,
        'a': 0.95,
        'init_epoch': 0
    },
    'lr_decay_f': exp_decay,

    # Reduce learning rate on plateau
    'on_plateau': True,
    'on_plateau_kwargs': {
        'monitor': 'val_loss',
        'factor': 0.1,
        'patience': 3,
        'verbose': 1
    },

    'cache': cache,

#   'pretrained_model': 'load_best',
#   'pretrained_model': os.path.join(GENERATED_DATA, "weights", ""),

    'output_path': OUTPUT_PATH,
}

params['save_prefix_template'] = '{cnn_name}_tag=%i_fold={fold_index}_seed=%i' % (0, params['seed'])
params['input_shape'] = params['image_size'] + (7,)
params['n_classes'] = 1

In [8]:
from data_utils import get_label
labels = np.zeros((len(trainval_id_type_list), 1), dtype=np.uint8)
for i, it in enumerate(trainval_id_type_list):
    labels[i, 0] = get_label(*it, tag=tag)

In [9]:
stats = np.sum(labels, axis=0)
stats, len(trainval_id_type_list)

(array([339], dtype=uint64), 675)

In [10]:
# Start CV

n_folds = 5
val_fold_index = 0
val_fold_indices = []  # !!! CHECK BEFORE LOAD TO FLOYD
hists = []

kf = KFold(n_splits=n_folds)
trainval_id_type_list = np.array(trainval_id_type_list)
for train_index, test_index in kf.split(trainval_id_type_list):
    train_id_type_list, val_id_type_list = trainval_id_type_list[train_index], trainval_id_type_list[test_index]

    if len(val_fold_indices) > 0:
        if val_fold_index not in val_fold_indices:
            val_fold_index += 1
            continue

    val_fold_index += 1
    print("\n\n ---- Validation fold index: ", val_fold_index, "/", n_folds)

    print(datetime.now(), len(train_id_type_list), len(val_id_type_list))
    assert len(to_set(train_id_type_list) & to_set(val_id_type_list)) == 0, "WTF"

    weights = None if 'pretrained_model' in params else None
    cnn = params['network'](lr=params['lr_kwargs']['lr'], weights=weights, **params)
    params['save_prefix'] = params['save_prefix_template'].format(cnn_name=cnn.name, fold_index=val_fold_index - 1)
    print("\n {} - Loaded {} model ...".format(datetime.now(), cnn.name))

    if 'pretrained_model' in params:
        load_pretrained_model(cnn, **params)

    print("\n {} - Start training ...".format(datetime.now()))
    h = train(cnn, train_id_type_list, val_id_type_list, **params)
    if h is None:
        continue
    hists.append(h)



 ---- Validation fold index:  1 / 5
2017-07-18 02:17:52.748380 540 135

 2017-07-18 02:17:55.078104 - Loaded Separable_SqueezeNet_BN_on_tif model ...

 2017-07-18 02:17:55.078213 - Start training ...

-- Training parameters: 32, 25, 544, 160

-- Fit model

-- Fit stats of train dataset
Load existing file: /Users/vfomin/Documents/ML/Kaggle/PlanetAmazonRainForest/common/../output/generated/Separable_SqueezeNet_BN_on_tif_tag=0_fold=0_seed=2017_stats.npz
No need to recompute statistics
- New Keras API found -
Epoch 1/25
16/17 [===========================>..] - ETA: 1s - loss: 0.7391 - precision: 0.2483 - recall: 0.1319
Epoch validation: f2 = 1.000000, mae=0.000000 

17/17 [==============================] - 44s - loss: 0.7358 - precision: 0.2337 - recall: 0.1241 - val_loss: 0.8402 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/25
17/17 [==============================] - 28s - loss: 0.6615 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.9857 - val_precision: 0.0000

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-45514b3ff6c4>", line 25, in <module>
    cnn = params['network'](lr=params['lr_kwargs']['lr'], weights=weights, **params)
  File "/Users/vfomin/Documents/ML/Kaggle/PlanetAmazonRainForest/common/models/squeezenet_tif_classification.py", line 59, in get_squeezenet_on_tif
    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
  File "/Users/vfomin/Documents/ML/Kaggle/PlanetAmazonRainForest/common/models/squeezenet_tif_classification.py", line 34, in fire_module
    left = BatchNormalization(name=s_id + bn + exp1x1)(left)
  File "/usr/local/lib/python3.5/site-packages/keras/engine/topology.py", line 569, in __call__
    self.build(input_shapes[0])
  File "/usr/local/lib/python3.5/site-packages/keras/layers/normalization.py", line 123, in build
    trainable=False)
  File 

KeyboardInterrupt: 

In [11]:
from training_utils import get_gen_flow, get_val_imgaug_seq
from metrics import score

In [12]:
train_gen, train_flow = get_gen_flow(id_type_list=train_id_type_list,
                                     imgaug_seq=params['train_seq'](params['seed']),
                                     **params)
params2 = dict(params)
params2['normalization'] = 'from_save_prefix'

val_gen, val_flow = get_gen_flow(id_type_list=val_id_type_list,
                                 imgaug_seq=get_val_imgaug_seq(params['seed']),
                                 **params2)


-- Fit stats of train dataset
Load existing file: /Users/vfomin/Documents/ML/Kaggle/PlanetAmazonRainForest/common/../output/generated/Separable_SqueezeNet_BN_on_tif_tag=0_fold=0_seed=2017_stats.npz
No need to recompute statistics


In [13]:
for x, y in train_flow:
    break

In [16]:
y_pred.shape

(32, 1)

In [17]:
y_pred = cnn.predict_on_batch(x)
print((y_pred > 0.35).astype(np.uint8) - 0.77 * y), 
print(score(y, (y_pred > 0.35).astype(np.uint8)))

[[ 1.  ]
 [ 1.  ]
 [ 1.  ]
 [ 1.  ]
 [ 1.  ]
 [ 1.  ]
 [ 0.23]
 [ 0.23]
 [ 1.  ]
 [ 0.23]
 [ 0.23]
 [ 0.23]
 [ 1.  ]
 [ 0.23]
 [ 1.  ]
 [ 0.23]
 [ 0.23]
 [ 0.23]
 [ 0.23]
 [ 0.23]
 [ 0.23]
 [ 1.  ]
 [ 0.23]
 [ 1.  ]
 [ 1.  ]
 [ 0.23]
 [ 0.23]
 [ 1.  ]
 [ 1.  ]
 [ 1.  ]
 [ 1.  ]
 [ 0.23]]
0.833333333333


In [35]:
for x2, y2 in val_flow:
    break

In [37]:
y_pred = cnn.predict_on_batch(x2)
print((y_pred > 0.35).astype(np.uint8) - 0.77 * y2), 
print(score(y2, (y_pred > 0.1).astype(np.uint8)))

[[ 1.    0.    0.   -0.77  0.    0.    0.  ]
 [ 0.    0.   -0.77  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 1.    0.    0.   -0.77  0.    0.    0.  ]
 [ 1.    0.    0.   -0.77  0.    0.    0.  ]
 [ 0.   -0.77  0.    0.    0.    0.    0.  ]
 [ 0.    0.   -0.77  0.    0.    0.    0.  ]
 [ 0.   -0.77  0.    0.    0.    0.    0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 1.    0.    0.    0.    0.   -0.77  0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 1.    0.    0.    0.    0.   -0.77  0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 1.    0.    0.   -0.77  0.    0.    0.  ]
 [ 0.   -0.77  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.77  0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.77  0.  ]
 [ 1.    0

In [ ]:
# ### Validation all classes

n_runs = 2
n_folds = 5
run_counter = 0
cv_mean_scores = np.zeros((n_runs, n_folds))
val_fold_indices = []  # !!! CHECK BEFORE LOAD TO FLOYD

params['pretrained_model'] = 'load_best'

_trainval_id_type_list = np.array(trainval_id_type_list)

while run_counter < n_runs:
    run_counter += 1
    print("\n\n ---- New run : ", run_counter, "/", n_runs)
    val_fold_index = 0
    kf = KFold(n_splits=n_folds)
    for train_index, test_index in kf.split(_trainval_id_type_list):
        train_id_type_list, val_id_type_list = _trainval_id_type_list[train_index], _trainval_id_type_list[test_index]

        if len(val_fold_indices) > 0:
            if val_fold_index not in val_fold_indices:
                val_fold_index += 1
                continue

        val_fold_index += 1
        print("\n\n ---- Validation fold index: ", val_fold_index, "/", n_folds)

        print(len(train_id_type_list), len(val_id_type_list))
        assert len(to_set(train_id_type_list) & to_set(val_id_type_list)) == 0, "WTF"

        cnn = params['network'](input_shape=params['input_shape'], n_classes=params['n_classes'])
        params['save_prefix'] = params['save_prefix_template'].format(cnn_name=cnn.name, fold_index=val_fold_index-1)
        print("\n {} - Loaded {} model ...".format(datetime.now(), cnn.name))

        load_pretrained_model(cnn, **params)

        params['seed'] += run_counter - 1

        f2, mae = validate(cnn, val_id_type_list, verbose=0, **params)
        cv_mean_scores[run_counter-1, val_fold_index-1] = f2

        np.random.shuffle(_trainval_id_type_list)

print(cv_mean_scores)
